In [23]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

In [24]:
def load_array(data_arrays, batch_size, is_train=True): #@save
    """构造⼀个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays) #列表入参，输入两个数据一一对应
    return data.DataLoader(dataset, batch_size,shuffle=is_train)#重新排序

In [25]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [26]:
# nn是神经⽹络的缩写
from torch import nn
net = nn.Sequential(nn.Linear(2, 1))
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [27]:
# 定义损失函数
loss = nn.MSELoss()
# 定义优化算法
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [28]:
# 训练
'''
• 通过调⽤net(X)⽣成预测并计算损失l（前向传播）。
• 通过进⾏反向传播来计算梯度。
• 通过调⽤优化器来更新模型参数。
'''
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step() #更新权重和偏差
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000204
epoch 2, loss 0.000097
epoch 3, loss 0.000097


In [29]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([0.0003, 0.0011])
b的估计误差： tensor([-0.0001])


⼩结    
• 我们可以使⽤PyTorch的⾼级API更简洁地实现模型。   
• 在PyTorch中，data模块提供了数据处理⼯具，nn模块定义了⼤量的神经⽹络层和常⻅损失函数。   
• 我们可以通过_结尾的⽅法将参数替换，从而初始化参数。

练习
1. 如果将小批量的总损失替换为小批量损失的平均值，你需要如何更改学习率？
2. 查看深度学习框架⽂档，它们提供了哪些损失函数和初始化⽅法？⽤Huber损失代替原损失，即
$$
l\left(y, y^{\prime}\right)= \begin{cases}\left|y-y^{\prime}\right|-\frac{\sigma}{2} & \text { if }\left|y-y^{\prime}\right|>\sigma \\ \frac{1}{2 \sigma}\left(y-y^{\prime}\right)^{2} & \text { 其它情况 }\end{cases}
$$
3. 你如何访问线性回归的梯度？

1. 将学习率除以batchsize

In [ ]:
#2. 其实Huber损失可以用torch自带的函数解决：
# torch.nn.SmoothL1Loss()
import torch.nn as nn
import torch.nn.functional as F
class HuberLoss(nn.Module):
    def __init__(self, sigma):
        super(HuberLoss, self).__init__()
        self.sigma = sigma
    def forward(self, y, y_hat):
        if F.l1_loss(y, y_hat) > self.sigma:
            loss = F.l1_loss(y, y_hat) - self.sigma/2
        else:
            loss = (1/(2*self.sigma))*F.mse_loss(y, y_hat)
        return loss

In [30]:
a = net[0].weight.grad
b = net[0].bias.grad
a,b
# 假如是多层网络，可以用一个self.xxx=某层，然后在外面通过net.xxx.weight.grad和net.xxx.bias.grad把梯度给拿出来。

(tensor([[0.0008, 0.0096]]), tensor([-0.0103]))